\begin{titlepage}

% Photo of FS
\centering
\vspace{-40pt}
\includegraphics[width=0.8\textwidth]{Frankfurt_School_Logo.jpg}\par
\vspace{2.5cm}

% Course
{\scshape\huge Quantitative Trading\par}
{\scshape\huge and Analysis with Python \par}
\vspace{2.5cm}

% Title
{\Huge\bfseries Report \par}
{\scshape\large Course Project \par}

\vspace{2cm} % If signature is taken might have to add space.

{\Large Jiawei Li (8448317)\par}
{\Large Sebastian Sydow (8316620)\par}
{\Large Strahinja Trenkic (8441963)\par}
{\Large Xiaoyi Zhou (8419669)\par}

\vspace{0.5cm}

% Date
\vfill
{\large \today\par}
\end{titlepage}


\newpage

\hypersetup{linkcolor=black}
\tableofcontents

\newpage

# Introduction

This report accompanies the Jupyter Notebook submitted for the course project in the course *Quantitative Trading and Analysis with Python* (M.Sc.) at the Frankfurt School of Finance and Management. This report includes:

- Brief summary of our approach

- Summary of the the final results of our strategy. 

# Summary of the Approach

## Data Sources and Retrieval
For our course project, we used different data sources (e.g. Quandl, WRDS, St. Louis FED) out of which most were downloaded within the jupyter notebook submitted alongside this report, e.g. daily returns and stock prices from WRDS (CRSP). Nevertheless, in order to run the notebook from top to bottom, one needs three additional data files:

- Selected stocks for this assignment stored in `df_input_stocks`, retrieved from Canvas.

- Generalized lower bounds for the expected excess simple returns (`df_glb`). This data was provided by [Grigory Vilkov](https://www.vilkov.net/index.html) and retrieved from [here](https://osf.io/2gkcw/).

- Risk-neutral skewness (`df_mfis`), again provided by [Grigory Vilkov](https://www.vilkov.net/index.html) and retrieved from [here](https://osf.io/a2g3h/).

## Factors Employed in our Models
After pre-processing the data, i.e. dealing with non-values, analysis of multicollinearity and data normalization, we experimented with a variety of factors. Our final model utilized the following factors:

**Skew1:** 
The SKEW defined as the difference between OTM put [delta = -10] implied volatility and ATM call [delta = 50] implied volatility from the 30-day Volatility Surface File of OptionMetrics.

**Skew2:** 
The SKEW defined as the difference between OTM put [delta = -25] implied volatility and ATM call [delta = 25] implied volatility from the 30-day Volatility Surface File of OptionMetrics.

**Implied Volatility:**
The average volatility [AIV] of all OTM call and put options from the 30-day Volatility Surface File of OptionMetrics. 

**Relative Strength Index:**
We used the Relative Strength Index (RSI) to measure the magnitude of recent price changes. The RSI is a momentum indicator. 

**4-Factor-Model:** We downloaded (daily) data about the four-factor model from the [Ken French data library](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html) and estimated the $\beta$-factors for the four-factor model for each stock using a 1-year rolling window of daily data (i.e. 252 [trading] days). The $\beta$-factors were updated at the end of each month and used for the following month until the next update. The $\beta$-factor were estimated using statsmodels [rolling ordinary least squares](https://www.statsmodels.org/dev/generated/statsmodels.regression.rolling.RollingOLS.html). 

**Yield Curve / Yield Difference:** We downloaded data about U.S. Treasury yield curve rates from [quandl](https://data.nasdaq.com/data/USTREASURY/YIELD-treasury-yield-curve-rates). We calculated the difference between the 10-year yield and the 1-year yield to construct a factor storing information about the Treasury yield curve (term structure). As the yield-difference is not stock specific, i.e. observable for each stock, we also calculated the $\beta$-factor for each stock. Again, we used a 1-year rolling window of daily data (i.e. 252 [trading] days) and updated the $\beta$-factors at the end of each month.

**Variance Risk Premium:** 
We constructed variance risk premium (VRP) on each day as the implied variance minus realized variance over the last month (21 [trading] days).

**AroonInd:**
The Aroon indicator was used to identify trend changes in the price of a stock, including the strength of that trend. 

## Implementation of Contraints
The task was to incorporate three constraints into our strategy. We implemented these constraints using the function `configure_inputs_for_optimisation`. In doing so, we ensured that the exposure of the previously outlined factors deviates max. 5 % compared to the benchmark, the weight of each stock deviates max. 10% compared to the benachmark and the drawdown relative to the benchmark is max. 1% per month. For the code of our implementation, please refer to the function `configure_inputs_for_optimisation` in the Jupyter Notebook accompanying this report.

## Model

We used three models for return prediction:

- Random Forest
- Lasso Regression
- XGBoost


## Strategy

We implemented two strategies, which we also backtested:

- MSR: Maximize-Sharpe-Ratio

- MVP: Minimize-Variance-Portfolio

# Final Results of our Strategy

On the following pages, the result of our two selected strategies compared to the benchmark are presented, including statistics such as Sharp Ratio and Cumulative Return. They are trained with different factors and optimizors, and both strategies beat the benchmark as can be seen by a Sharpe Ratio of 1.18 compared to 1.14 (the Random Forest model) and a Sharp Ratio of 1.24 compared to 1.21 (the XGBoost model). Please note that a different starting time of the out-of-sample period is applied for two models and thus the Sharpe Ratio are different there. Nevertheless, due to strict constraints, one can identify that our strategy closely follows the performance of the benchmark and only slightly deviates over the entire timeframe.